# Assignment 1 for IE 306.02
In this example, a call center is simulated. Customers call the call center at random times and an automated answering machine process the incoming calls. After collecting the data of the customers, it routes these calls to operators. There is a probability that the answering machine routes the call to the wrong operator. If that happens, the caller hangs up immediately. 

After leaving the answering machine, the caller is routed to operator 1 or 2. If the operator is busy, the caller waits in queue. Callers renege after 10 minutes of waiting. When the caller is done, it leaves the system.

The operators can take 3 min break, if there are no customers waiting for them.

In [1]:
import simpy
import random
import numpy as np
from simpy.events import Event

Define global variables that the simulation needs. This includes key parameters for the interarrival, service time, answering machine service time, and the time that shifts last.

In [2]:
# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

# Every shift last 8 hours(480 minutes).
SHIFT = 480

Define the necessary variables and arrays for bookkeeping.

In [3]:
ANSWERED_CALLS = 0        # Total answered calls.
FINISHED_T = 0            # The time simulation ends.
all_customers = []        # The array of all answered customers.
operation_t = [0,0]       # Service times of operators.
shift_number = 0          # Keeps track of how many shifts have passed.
is_done = 0               # The number of customers that left the system.
unsatisfied_customers = 0 # The number of customers that leaves the system unsatisfied.

Define service time of operators.

In [4]:
def service(env, opr):
    if opr==operator1:
        time = random.lognormvariate(M,S)     # LogNormal distribution.
        operation_t[0] += time
        yield env.timeout(time)
        
    elif opr==operator2:
        time = random.uniform(*service_range) # Uniform distribution.
        operation_t[1] += time
        yield env.timeout(time)

Define the Customer class. Every time a customer calls, an instance is initiated. Every customer activates the "call" process.

* The automated answering machine processes the incoming call. After collecting the data of customer, it routes the customer to operator 1 or 2. The routing process may fail. In this case, customer leaves the system unsatisfied.

* The customer initiates a request for operator resource. If the operator is busy, the customer waits for the operator to be available. The customer gets the service and leaves the system. If a customer waits for 10 minutes, it leaves the system unsatisfied.

* The simulation terminates, when the targeted number of customers is reached.

In [5]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env     # Simulation environment.
        self.name = name   # Name of customer. It includes the number of customer.
        self.arrival_t = self.env.now # Arrival time of customer.
        self.action = env.process(self.call()) #call process is initiated.
        self.waiting_t = 0 # The time that this customer waits in queue.
        self.system_t = 0  # The time that this customer spends in the system.
        self.unsatisfied = False # Whether this customer is unsatisfied or not.
        
    # This customer initiates a call.
    def call(self):
        global is_done
        global unsatisfied_customers
        
        if router.is_idle(): # Collecting data for answering machine.
            router.working_t += [self.env.now]    
        router.count += [self.name]
        
        # Answering machine collects the data of this customer.
        yield self.env.timeout(random.expovariate(1/router_mean)) # Exponential distribution.
        
        router.count.remove(self.name)
        if router.is_idle(): # Collecting data for answering machine.
            router.idle_t += [self.env.now]
        
        # Routing this customer to an operators.
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            self.system_t = self.env.now - self.arrival_t
            is_done += 1
            unsatisfied_customers += 1
            self.unsatisfied = True
            if is_done == ANSWERED_CALLS:
                finished.succeed()
            return 
        
        # This customer initiates a request for operator resource.
        with self.operator.request(0) as req: # The priority of breaks is low.
            time = self.env.now
            result = yield req | env.timeout(10)
            
            # Reneging after 10 mins
            if req not in result:
                self.waiting_t = self.env.now - time
                self.system_t = self.env.now - self.arrival_t
                is_done += 1
                unsatisfied_customers += 1
                self.unsatisfied = True
                if is_done == ANSWERED_CALLS:
                    finished.succeed()
                return 
            
            # This customer is getting service.
            self.waiting_t = self.env.now - time
            yield self.env.process(service(env,self.operator))
            self.system_t = self.env.now - self.arrival_t
            is_done += 1
            if is_done == ANSWERED_CALLS:
                finished.succeed()

Define the answering machine class. The answering machine can serve 100 callers simultaneously. This class controls the availability of answering machine and collects the data for utilization.

* When the answering machine starts to work, the time is kept. It can serve multiple customers at the same time. When there is no customer, it becomes idle and this time is also kept. Using these data, the utilization of the answering machine is calucalted.

In [6]:
class robocall:
    def __init__(self,env):
        self.count = []     # Keeps the customers.
        self.working_t = [] # Keeps the time answering machine starts to work.
        self.idle_t = []    # Keeps the time answering machine becomes idle.
        self.env = env      # Simulation environment.
        
    # Controls the availability of answering machine.
    def answer(self):
        return len(self.count) < 100
    def is_idle(self):
        return len(self.count) == 0
        
    # Calculates the utilization of this answering machine.
    def utilization(self):
        return sum([i-w for i,w in zip(router.idle_t, router.working_t)]) / FINISHED_T

Define the number of breaks and corresponding break times. 

In [7]:
def breaks():
    # Break numbers are distributed according to Poisson distribution.
    return [random.uniform(0,SHIFT) for i in range(np.random.poisson(8))]

Define shifts. When a new shift starts, this function is processed. The number of breaks and corresponding break times changes. For every break time, a new process is initiated.

In [8]:
def shift(env):
    global shift_number
    shift_number += 1
    
    # Creating process for each break time.
    [env.process(take_a_break(env, b, operator1)) for b in breaks()]
    [env.process(take_a_break(env, b, operator2)) for b in breaks()]

    yield env.timeout(SHIFT)

    env.process(shift(env))

Define breaks of an operator. When a new shift starts, new processes for break times are initiated. If the break time has come, a new request is initiated for operator resource. When there is no customers waiting for the operator, the operator takes a 3 min break. The breaks never affect the next shift.

In [9]:
def take_a_break(env, number, opr):
    yield env.timeout(number)
    with opr.request(1) as brk: # The priority of breaks is low.
        yield brk
        if SHIFT*shift_number - env.now > 0:
            if SHIFT*shift_number - env.now >= 3:
                yield env.timeout(3)
            else:
                yield env.timeout(480*shift_number - env.now)

Define the customer generator. This function keeps generating customers randomly, until the targeted number of customers reached. If the answering machine is not available, the customers cannot be generated. It can be said that the calls drop immediately.

In [10]:
def customer_generator(env):
    start = True
    num_answered = 0;
    
    while(len(all_customers)<ANSWERED_CALLS):
        
        # Starts the first shift.
        if start:
            env.process(shift(env))
            start = False
            
        yield env.timeout(random.expovariate(1/interarrival_mean)) # Exponential distribuiton.
        
        # Voice recognition system limit.
        if router.answer():
            num_answered += 1
            customer = Customer('Customer %s' %(num_answered), env)
            all_customers.append(customer)
            
    FINISHED_T = env.now 

Define necessary simulation elements.

* The operators give priority to customers.

In [11]:
env = simpy.Environment() # Simulation environment.

finished = Event(env)     # Controls the termination of simulation.
router = robocall(env)    # The answering machine.

# The operators in the system. 
# Because the priorities of breaks and customers are different, PriorityResource is used.
operator1 = simpy.PriorityResource(env, capacity = 1)
operator2 = simpy.PriorityResource(env, capacity = 1)

Define simulation function for multiple simulations at once.

In [12]:
def simulation(CALLS, seed):
    
    global FINISHED_T 
    global all_customers
    global operation_t
    global shift_number
    global ANSWERED_CALLS
    global is_done
    global unsatisfied_customers
    
    unsatisfied_customers = 0
    is_done = 0
    ANSWERED_CALLS = CALLS
    FINISHED_T = 0
    all_customers = []
    operation_t = [0,0]
    shift_number = 0
    random.seed(seed)
    
    global  env 
    global finished 
    global router 
    global operator1 
    global operator2 
    env = simpy.Environment()
    finished = Event(env)
    router = robocall(env)
    operator1 = simpy.PriorityResource(env, capacity = 1)
    operator2 = simpy.PriorityResource(env, capacity = 1)
    env.process(customer_generator(env))
    env.run(finished) 
    FINISHED_T = env.now
    
    # Output data.
    customer_waiting_1 = [c.waiting_t for c in all_customers if c.operator == operator1]
    customer_waiting_2 = [c.waiting_t for c in all_customers if c.operator == operator2]
    customer_number_1 = [1 for waiting_time in customer_waiting_1 if waiting_time != 0]
    customer_number_2 = [1 for waiting_time in customer_waiting_2 if waiting_time != 0]
    unsatisfied_system_time = [c.system_t for c in all_customers if c.unsatisfied == True]
    system_time = [c.system_t for c in all_customers]
        
    return (router.utilization(),  
            operation_t[0]/FINISHED_T, 
            operation_t[1]/FINISHED_T,
            (operation_t[0]+operation_t[1])/FINISHED_T,
            (sum(customer_waiting_1)+sum(customer_waiting_2))/ANSWERED_CALLS,
            (sum(customer_waiting_1)+sum(customer_waiting_2))/sum(system_time),
            sum(customer_waiting_1)/FINISHED_T,
            sum(customer_number_1),
            sum(customer_waiting_2)/FINISHED_T,
            sum(customer_number_2),
            sum(unsatisfied_system_time)/FINISHED_T,
            unsatisfied_customers)

In [13]:
def printStatistics(array, calls):
    print("Statistics for", calls, "callers:\n")
    sums = [0 for c in range(12)]
    for i in array:
        for j in range(len(i)):
            sums[j] += i[j]
            if j == 0:
                print("Utilization of the answering system:", i[j])
            elif j == 1:
                print('Utilization Operator 1:', i[j], 
                  '\nUtilization Operator 2:' , i[j+1],
                  '\nUtilization of Both Operators:', i[j+2])
            elif j == 4:
                print("Average Total Waiting Time:", i[j])
            elif j == 5:
                print("Maximum Total Waiting Time to Total System Time Ratio:", i[j])
            elif j == 6:
                print("Time-average number of people waiting to be served by Operator 1:", i[j],
                     "\nNumber of people waited to be served by Operator 1:", i[j+1])
            elif j == 8:
                print("Time-average number of people waiting to be served by Operator 2:", i[j],
                     "\nNumber of people waited to be served by Operator 2:", i[j+1])
            elif j == 10:
                print("Time-average number of unsatisfied customers in the system:", i[j],
                    "\nNumber of unsatisfied customers:", i[j+1])
        print()
    average = [s / 10 for s in sums]
    print("Average of 10 simulations:\n")
    print("Utilization of the answering system:", average[0])
    print('Utilization Operator 1:', average[1], 
                  '\nUtilization Operator 2:' , average[2],
                  '\nUtilization of Both Operators:', average[3])
    print("Average Total Waiting Time:", average[4])
    print("Maximum Total Waiting Time to Total System Time Ratio:", average[5])
    print("Time-average number of people waiting to be served by Operator 1:", average[6],
                     "\nNumber of people waited to be served by Operator 1:", average[7])
    print("Time-average number of people waiting to be served by Operator 2:", average[8],
                     "\nNumber of people waited to be served by Operator 2:", average[9])
    print("Time-average number of unsatisfied customers in the system:", average[10],
                    "\nNumber of unsatisfied customers:", average[11])
    
    variance = [0 for c in range(12)]
    for i in array:
        for j in range(len(i)):
            variance[j] += (i[j] - average[j])**2
    variance[:] = [v / 9 for v in variance]
    print("\nVariance of 10 simulations:\n")
    print("Utilization of the answering system:", variance[0])
    print('Utilization Operator 1:', variance[1], 
                  '\nUtilization Operator 2:' , variance[2],
                  '\nUtilization of Both Operators:', variance[3])
    print("Average Total Waiting Time:", variance[4])
    print("Maximum Total Waiting Time to Total System Time Ratio:", variance[5])
    print("Time-average number of people waiting to be served by Operator 1:", variance[6],
                     "\nNumber of people waited to be served by Operator 1:", variance[7])
    print("Time-average number of people waiting to be served by Operator 2:", variance[8],
                     "\nNumber of people waited to be served by Operator 2:", variance[9])
    print("Time-average number of unsatisfied customers in the system:", variance[10],
                    "\nNumber of unsatisfied customers:", variance[11])

In [14]:
%%time
seeds = [305, 451, 973, 867, 135, 231, 53, 199, 155, 747]
calls = [1000, 5000]
statistics_1000 = [simulation(1000,s) for s in seeds]
statistics_5000 = [simulation(5000,s) for s in seeds]

Wall time: 6.28 s


In [15]:
printStatistics(statistics_1000, calls[0])

Statistics for 1000 callers:

Utilization of the answering system: 0.5808244997253904
Utilization Operator 1: 0.4463594996354793 
Utilization Operator 2: 0.40730852447275334 
Utilization of Both Operators: 0.8536680241082327
Average Total Waiting Time: 2.04348930996295
Maximum Total Waiting Time to Total System Time Ratio: 0.16811228947421036
Time-average number of people waiting to be served by Operator 1: 0.1744201890990934 
Number of people waited to be served by Operator 1: 157
Time-average number of people waiting to be served by Operator 2: 0.1767904168246609 
Number of people waited to be served by Operator 2: 280
Time-average number of unsatisfied customers in the system: 0.26311849272222454 
Number of unsatisfied customers: 161

Utilization of the answering system: 0.5612436823593874
Utilization Operator 1: 0.4396799411618331 
Utilization Operator 2: 0.38458098762666876 
Utilization of Both Operators: 0.8242609287885019
Average Total Waiting Time: 1.9243084226276395
Maximum To

In [16]:
printStatistics(statistics_5000, calls[1])

Statistics for 5000 callers:

Utilization of the answering system: 0.5649549112453646
Utilization Operator 1: 0.44922593170752995 
Utilization Operator 2: 0.41284087623985055 
Utilization of Both Operators: 0.8620668079473804
Average Total Waiting Time: 1.8906015093968875
Maximum Total Waiting Time to Total System Time Ratio: 0.1584731566134308
Time-average number of people waiting to be served by Operator 1: 0.1396224354136161 
Number of people waited to be served by Operator 1: 697
Time-average number of people waiting to be served by Operator 2: 0.1796821760580851 
Number of people waited to be served by Operator 2: 1460
Time-average number of unsatisfied customers in the system: 0.1904505542589893 
Number of unsatisfied customers: 696

Utilization of the answering system: 0.564909639945092
Utilization Operator 1: 0.4231581854565804 
Utilization Operator 2: 0.4140930143125253 
Utilization of Both Operators: 0.8372511997691057
Average Total Waiting Time: 1.818325208175282
Maximum Tot